In [1]:
import solana
from solana.publickey import PublicKey
from solana.rpc.api import Client
from solana.keypair import Keypair
from solana.system_program import SYS_PROGRAM_ID
from solana import system_program
from solana import transaction
from solana.sysvar import SYSVAR_RENT_PUBKEY, SYSVAR_CLOCK_PUBKEY
from solana.transaction import *
from spl.token import constants
from spl.token import instructions as assoc_instructions
import json
import jsonrpcclient
from solana.rpc.core import RPCException
# import sys
# sys.path.append('/instruction')
from instruction import *
from state import Constants as ingl_constants
client = Client("http://api.devnet.solana.com")
client.is_connected()

True

In [2]:
payer_keypair = Keypair.from_seed(b"B2JeG8Dft3pBfQ4cX3bSqmJUFg11kD5h")
payer_keypair.public_key 

2aphfLATTWqoB5jqYqDfeFBkkKDncpwysDHvdCgQgDFe

In [3]:
def create_collection(payer_keypair):
    mint_pubkey, _mint_pubkey_bump = PublicKey.find_program_address([bytes(ingl_constants.INGL_NFT_COLLECTION_KEY, 'UTF-8')], ingl_constants.INGL_PROGRAM_ID)
    mint_authority_pubkey, _mint_authority_pubkey_bump = PublicKey.find_program_address([bytes(ingl_constants.INGL_MINT_AUTHORITY_KEY, 'UTF-8')], ingl_constants.INGL_PROGRAM_ID)
    minting_pool_pubkey, _minting_pool_pubkey_bump = PublicKey.find_program_address([bytes(ingl_constants.INGL_MINTING_POOL_KEY, 'UTF-8')], ingl_constants.INGL_PROGRAM_ID)
    collection_holder_pubkey, _collection_holder_pubkey_bump = PublicKey.find_program_address([bytes(ingl_constants.COLLECTION_HOLDER_KEY, 'UTF-8')], ingl_constants.INGL_PROGRAM_ID)
    mint_associated_account_pubkey = assoc_instructions.get_associated_token_address(collection_holder_pubkey, mint_pubkey)
    token_metadata_pubkey = PublicKey("metaqbxxUerdq28cj1RbAWkYQm3ybzjb6a8bt518x1s")
    metadata_pda, _metadata_pda_bump = PublicKey.find_program_address([b"metadata", bytes(token_metadata_pubkey), bytes(mint_pubkey)], token_metadata_pubkey)
    master_edition_pda, _master_edition_bump = PublicKey.find_program_address([b"metadata", bytes(token_metadata_pubkey), bytes(mint_pubkey), b"edition"], token_metadata_pubkey)
    global_gem_pubkey, _global_gem_bump = PublicKey.find_program_address([bytes(ingl_constants.GLOBAL_GEM_KEY, 'UTF-8')], ingl_constants.INGL_PROGRAM_ID)

    payer_account_meta = AccountMeta(payer_keypair.public_key, True, True)
    collection_holder_meta = AccountMeta(collection_holder_pubkey, False, True) #This might be the cause of a Writable escalated permission error.
    mint_account_meta = AccountMeta(mint_pubkey, False, True)
    mint_authority_meta = AccountMeta(mint_authority_pubkey, False, False)
    mint_associated_meta = AccountMeta(mint_associated_account_pubkey, False, True)
    spl_program_meta = AccountMeta(constants.TOKEN_PROGRAM_ID, False, False)
    sysvar_rent_account_meta = AccountMeta(solana.sysvar.SYSVAR_RENT_PUBKEY, False, False)
    system_program_meta = AccountMeta(system_program.SYS_PROGRAM_ID, False, False)
    token_metadata_meta = AccountMeta(metadata_pda, False, True)
    metadata_program_id = AccountMeta(token_metadata_pubkey, False, False)
    associated_program_meta = AccountMeta(constants.ASSOCIATED_TOKEN_PROGRAM_ID, False, False)
    global_gem_meta = AccountMeta(global_gem_pubkey, False, True)
    edition_meta = AccountMeta(master_edition_pda, False, True)

    accounts = [
        payer_account_meta,
        collection_holder_meta,
        mint_account_meta,
        mint_authority_meta,
        mint_associated_meta,
        spl_program_meta,
        sysvar_rent_account_meta,
        system_program_meta,
        token_metadata_meta,
        global_gem_meta,
        edition_meta,

        system_program_meta,
        system_program_meta,
        spl_program_meta,
        associated_program_meta,
        spl_program_meta,
        metadata_program_id,
        metadata_program_id,
    ]
    data = build_instruction("MintNewCollection")
    transaction = Transaction()
    transaction.add(TransactionInstruction(accounts, ingl_constants.INGL_PROGRAM_ID, data))
    return f"https://explorer.solana.com/tx/{client.send_transaction(transaction, payer_keypair)['result']}?cluster=devnet"


create_collection(payer_keypair)

'https://explorer.solana.com/tx/4XepMdP3483PkbRpYPeLCHEdvcV85uHLetJhmNexBRMs7cTWvspePvDdGge1at3G1HsK7uqqoobEY8v5HN9Y35um?cluster=devnet'